# 🧠 Dream Travel Recommender – psycopg2 Setup
This notebook connects to your PostgreSQL database using `psycopg2`, fetches destination data, and prepares it for building a content-based recommender system.

In [ ]:
import pandas as pd
import psycopg2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# 🔐 Replace these credentials with your actual values
DB_NAME = "travel_db"
DB_USER = "postgres"
DB_PASS = "post@Charade@01"
DB_HOST = "127.0.0.1"
DB_PORT = "5432"

try:
    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        host=DB_HOST,
        port=DB_PORT
    )
    print("✅ Connected to PostgreSQL!")
except Exception as e:
    print("❌ Connection failed.")
    raise e


In [ ]:
# Read destination data into pandas
query = """
SELECT id, name, city, state, country, description, tags,
       adventure, relax, nature, culture, luxury
FROM destinations
"""
df = pd.read_sql(query, conn)
print(f"Loaded {len(df)} records.")
df.head()


In [ ]:
# Combine description and tags into a single text column
df["tags_str"] = df["tags"].apply(lambda x: " ".join(x) if isinstance(x, list) else str(x))
df["text"] = df["description"].fillna("") + " " + df["tags_str"].fillna("")
df["text"] = df["text"].str.lower()
df["text"].head()


In [ ]:
# Fit TF-IDF model on text
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["text"])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix)
print("✅ TF-IDF matrix shape:", tfidf_matrix.shape)


In [ ]:
def recommend_places(query_index, top_n=10):
    sim_scores = list(enumerate(cosine_sim[query_index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    recommended = [df.iloc[i[0]][["name", "country", "description"]] for i in sim_scores]
    return pd.DataFrame(recommended)


In [ ]:
# 🔍 Test with any destination index
recommend_places(0, top_n=5)
